In [2]:
'''
Sample code to read dicom images and place slices as images for deep learning
'''
import matplotlib.pyplot as plt
import matplotlib.patches as patches  #subplot bounding boxes
from matplotlib.patches import Rectangle
import numpy as np
import os
import pydicom
from PIL import Image #for RGB manipulation
import gc #garbage collection used when running many plots


#MHA support files
import SimpleITK as sitk  #for reading the MHA files
from medpy.io import load #for mha files
import pickle

from copy import copy


def read_MHA(mha_full_path,mha_key):

    #####################################################
    #look for the MHA file. This will detail the 
    #segmented pixels for our label. The .mha files are 
    #stored above the \images folder
    ####################################################
    files_found = os.listdir(os.path.join(mha_full_path[mha_key], '..'))
    full_mha_file =''
    mha_found =0;
    for top_level_file in files_found:
        if top_level_file.endswith('.mha'):
            mha_name = top_level_file
            full_mha_file = os.path.join(mha_full_path[mha_key], '..', mha_name)
    if (full_mha_file == ''):
        print('!!!!! No MHA file found !!!!! @ ',mha_full_path[mha_key])
        mha_array=[]
    else:
        mha_array =get_mha_info(full_mha_file)
        mha_found = 1 #flag that we can use this later
    
    return mha_array, mha_found
    
    
    #open the mha file
    #itkimage = sitk.ReadImage(mha_full_file_path)
    #convert itkimage to 3D array
    #mha_data = sitk.GetArrayFromImage(itkimage)
    # the code below is for write the 3D array to a output file
    #return mha_data


def get_mha_info(mha_full_file_path):
    #open the mha file
    itkimage = sitk.ReadImage(mha_full_file_path)
    #convert itkimage to 3D array
    mha_data = sitk.GetArrayFromImage(itkimage)
    # the code below is for write the 3D array to a output file
    return mha_data 
    


def make_images():
    for ii in range(0,len(position)-1):
    #slice = list(newpos.keys())[ii]
    #print(slice)
    #a = next(iter(newpos.values()))
    #print(a)
    
        slice = list(newpos.keys())[ii-1]
        print(slice)
        print(newpos[slice])
        full_file_r = os.path.join(dicom_dir, slice)
        slice = list(newpos.keys())[ii]
        full_file_g = os.path.join(dicom_dir, slice)
        slice = list(newpos.keys())[ii+1]
        full_file_b = os.path.join(dicom_dir, slice)

        #dicom_data_r = pydicom.dcmread(full_file_r)
        #dicom_data_g = pydicom.dcmread(full_file_g)
        #dicom_data_b = pydicom.dcmread(full_file_b)


        dicom_data_r = pydicom.dcmread(full_file_r)
        dicom_data_g = pydicom.dcmread(full_file_r)
        dicom_data_b = pydicom.dcmread(full_file_r)

        #(dicom_data.PixelData)
        #full_file = os.join(dicom_dir,ii)
        #dicom_data = pydicom.dcmread(full_file)
        # r, g, and b are 512x512 float arrays with values >= 0 and < 1.
        rgbArray = np.zeros((cols,rows,3), 'uint8')
        print(len(dicom_data_g.PixelData))

        #sample display of image
        #plt.figure()
        #plt.imshow(dicom_data_r.pixel_array,cmap='gray')

        rgbArray[:,:, 0] = dicom_data_r.pixel_array
        rgbArray[:,:, 1] = dicom_data_g.pixel_array
        rgbArray[..., 2] = dicom_data_b.pixel_array
        img = Image.fromarray(rgbArray)


        #setup an image subdirectory for each patient. The patient folder
        #is stored under the patient_directory list
        #output_image_dir is the main folder to store images, but must have folder for each 
        #patient created
        patient_image_folder = os.path.join(output_image_dir,phase_name,patient_directories[0])
        rgb_file = os.path.join(output_image_dir, 'rgb' + str(ii) + '.png')
        print('rgb file is ',rgb_file)
        img.save(rgb_file)

        #position[ii]


###########################################################################
# Dicom top level for RCC data. We will need to drill down into this to get the individual patient records
# The directory structure is the default given in the requested data set. rcc_use will toggle between
# processing data under clear cell directories or under oncocytoma directories
rcc_use = 0
if (rcc_use ==0):
    top_data_dir = '/media/kgonzalez/Seagate Backup Plus Drive/Heidi/clear_cell_onco/clear_cell'
    output_image_dir = '/media/kgonzalez/Seagate Backup Plus Drive/PROJECT_PLOTS/RCC'
else:
    top_data_dir = '/media/kgonzalez/Seagate Backup Plus Drive/Heidi/clear_cell_onco/oncocytoma'
    output_image_dir = '/media/kgonzalez/Seagate Backup Plus Drive/PROJECT_PLOTS/ONCO'


#top_data_dir = onco_top_dir
#the clear cell directory is 
#  top
#     -> patient directories\
#          -> PreContrast\
#                      -> images\
#                               -> *.dcm
#              -> segmentation files (.mha,.roi)
#          -> post contrast nephro\
#          -> post coricomedullary\
#          -> post excretory\

# order will be patient_directories \ ct_type


#get list of patient directories underneath this
patient_directories = sorted(os.listdir(top_data_dir))
clear_cell_dict ={}
#for every patient, go into the folder and pull out the CT phase data sets
#patient_directory\
#                   ct_type1
#                   ct_type2
#                   ct_type3

ct_type= [] #list to hold the sub directories
counter = 0
print(type(patient_directories))
for ii in patient_directories:
    #print(ii)
    #get list of 4 sub directories
    ct_type.append(os.listdir(os.path.join(top_data_dir,ii)))
    #print('<>',ii,(os.listdir(os.path.join(top_data_dir,ii))))
    #store the ct type listing folders under the patient number key
    clear_cell_dict[ii] = os.listdir(os.path.join(top_data_dir,ii))



#for every patient, they have 4 folders of different tests conducted. Each folder will have the segmentation
#data in an .mha file



# Each patient folder has data from different phases of contrast propagation. 
# Run over each phase type for each patient (no contrast, post neph, excret)
#
for phase in range(3,4): #range(0,4):
    #
    # Get the image files in each directory
    #image_dict has the key being the top level patient folder, with the values being 
    #the various phases underneath the patient level
    #
    if (phase == 0):
        phase_name = 'Pre'
    elif(phase == 1):
        phase_name = 'Post_Cor'
    elif(phase == 2):
        phase_name = 'Post_Neph'
    elif(phase == 3):
        phase_name = 'Post_Exc'
    
    image_dict={}    #hold the image files
    for key, value in clear_cell_dict.items():

        image_dir = os.path.join(top_data_dir,key,value[phase])
        image_dir_listing = os.listdir(image_dir)
        image_files = os.listdir(os.path.join(image_dir,'images'))
        image_dict[key] = image_files


    full_path ={}
    for key, value in clear_cell_dict.items():
        pdir = key
        full_name = os.path.join(top_data_dir,
                                 pdir,
                                 clear_cell_dict[key][phase],
                                 'images')

        #check to make sure the directory exists
        if(os.path.isdir(full_name)== True):
            full_path[key] = full_name
        else:
            raise ValueError("DIRECTORY DOES NOT EXIST")


    #
    # Test options for debugginb
    #
    show_patient_info = 0 #list dicom patient specific tags
    patient_number =-1    #counter to list patient folders
    debug_folder = 0
    loop_counter = 0      #used to debug specific folders

    print('entering main section')
    for key, value in image_dict.items():

        #center on one folder to test changes
        if (debug_folder == 1):
            #debug specific folders
            key = '10024-0034_2003-12-05'
            value = image_dict[key]
            if (loop_counter > 0):
                print('loop counter is ',loop_counter)
            else:
                loop_counter = loop_counter + 1


        position = {}
        file_order = {}
        patient_number = patient_number + 1


        if (show_patient_info == 1):    print('KEY, VALUE:\n',key,value)
        if (show_patient_info == 1):    print('full path for this is \n',full_path[key])

        #get all of the dicom images stored under \images
        image_list = sorted(os.listdir(full_path[key]))  

        
        #check MHA annotation data
        mha_array, mha_found = read_MHA(full_path,key)


        for icount,ii in enumerate(image_list): #[0:20]:
            if (show_patient_info == 1):    print('key is ',key, 'icount is ',icount)

            full_file = os.path.join(full_path[key],ii)
            dicom_data = pydicom.dcmread(full_file)
            # Normal mode:
            if (show_patient_info == 1):    print("Filename.........:", full_file)
            if (show_patient_info == 1):    print("Storage type.....:", dicom_data.SOPClassUID)



            pat_name = dicom_data.PatientName
            display_name = pat_name.family_name + ", " + pat_name.given_name
            if (show_patient_info == 1):    print("Patient's name...:", display_name)
            if (show_patient_info == 1):    print("Patient id.......:", dicom_data.PatientID)
            if (show_patient_info == 1):    print("Modality.........:", dicom_data.Modality)
            if (show_patient_info == 1):    print("Study Date.......:", dicom_data.StudyDate)

            #check to make sure raw data is in the file
            if 'PixelData' in dicom_data:
                rows = int(dicom_data.Rows)
                cols = int(dicom_data.Columns)
                if (show_patient_info == 1):    print("Image size.......: {rows:d} x {cols:d}, {size:d} bytes".format(
                rows=rows, cols=cols, size=len(dicom_data.PixelData)))
            if 'PixelSpacing' in dicom_data:
                if (show_patient_info == 1):    print("Pixel spacing....:", dicom_data.PixelSpacing)

            # use .get() if not sure the item exists, and want a default value if missing
            print("Slice location...:", dicom_data.get('SliceLocation', "(missing)"))


            #assign slice position
            location = dicom_data.get('SliceLocation', "(missing)")
            position[icount] = location
            file_order[icount]= full_file

        #sort dictionary to get the slice order
        newpos = {}

        for keypos, valuepos in sorted(position.items(), key=lambda item: item[1]):
            #print("%s: %s" % (key, value))
            newpos[keypos] = valuepos



        #
        # MHA calculations
        #
        unique_slices=[]
        if(mha_found == 1):
            #get slices with lesion pixels
            #find any slice with values > 0.5 units

            lslice=[] #holds the slices containing the lesion values
            lesion_indices = np.argwhere(mha_array > 0.5)
            for xx in lesion_indices:
                lslice.append(xx[0])
            slice_numbers=set(lslice)
            unique_slices = list(slice_numbers)
            print(unique_slices)
                #now we have all the slices containing lesions, only print out those images
            lesion_middle_slice = int(np.median(unique_slices))



            nslice = mha_array[lesion_middle_slice,:,:]
            ldata = np.argwhere(nslice> 0.5)
            #print('ldata is ',ldata)
            #print('ldata \n',ldata[0])
            rowval=[]
            colval=[]
            for xx in ldata:
                #print(xx)
                rowval.append(xx[0])
                colval.append(xx[1])
            minrow = min(rowval)
            maxrow = max(rowval)
            mincol = min(colval)
            maxcol = max(colval)
            middlerow = int(np.floor((minrow + maxrow)/2))
            middlecol = int(np.floor((mincol + maxcol)/2))
            #bounding box will be +- 100 pixels each direction
        else:
            #no MHA was found in this set
            print('Skipping this dataset due to no annotation\n')
            continue #go to next iteration
        #########################################################################################
        #setup an image subdirectory for each patient. The patient folder
        #is stored under the patient_directory list
        #output_image_dir is the main folder to store images, but must have folder for each 
        #patient created

        idict_pos =2
        patient_image_folder = os.path.join(output_image_dir,
                                                phase_name,
                                                patient_directories[patient_number]) #image_dict[key][idict_pos])
        #print('patient folder --',patient_image_folder)
        
        if(os.path.isdir(patient_image_folder) == True):
            pass
        else:
            # Create directory
            try:
                # Create target Directory
                os.makedirs(patient_image_folder)
                print("Directory " , patient_image_folder ,  " Created ") 
            except OSError: #FileExistsError:
                print("Directory " , patient_image_folder ,  " failed to be created")
            else:
                pass            
        ###################################################################################
            

        for filenum in range(unique_slices[0],unique_slices[-1]): #range(0,len(position)-1):
            #slice = list(newpos.keys())[ii]
            #print(slice)
            #a = next(iter(newpos.values()))
            #print(a)
            print('filenumber is now ',filenum, ' out of ',len(position))

            #slice = list(newpos.keys())[filenum-1]
                #print(slice)
                #print(newpos[slice])
            full_file_r = file_order[filenum] #os.path.join(dicom_dir, slice)
            slice = list(newpos.keys())[filenum]
                #full_file_g = os.path.join(dicom_dir, slice)
                #slice = list(newpos.keys())[ii+1]
                #full_file_b = os.path.join(dicom_dir, slice)

                #dicom_data_r = pydicom.dcmread(full_file_r)
                #dicom_data_g = pydicom.dcmread(full_file_g)
                #dicom_data_b = pydicom.dcmread(full_file_b)


            dicom_data_r = pydicom.dcmread(full_file_r)
            dicom_data_g = pydicom.dcmread(full_file_r)
            dicom_data_b = pydicom.dcmread(full_file_r)

                #(dicom_data.PixelData)
                #full_file = os.join(dicom_dir,ii)
                #dicom_data = pydicom.dcmread(full_file)
                # r, g, and b are 512x512 float arrays with values >= 0 and < 1.
            #rgbArray = np.zeros((cols,rows,3), 'uint8')
            #zeropixels = np.zeros((cols,rows),'uint8')
            boxlength = 128
            subsetrows = boxlength
            subsetcols = boxlength
            rgbArray = np.zeros((subsetrows,subsetcols,3), 'uint8')
            zeropixels = np.zeros((subsetrows,subsetcols),'uint8')
                #print(len(dicom_data_g.PixelData))

                #sample display of image
                #plt.figure()
                #plt.imshow(dicom_data_r.pixel_array,cmap='gray')

            #rgbArray[:,:, 0] = dicom_data_r.pixel_array
            print('middle row, col\n',middlerow,middlecol)
            rgbArray[:,:, 0] = dicom_data_r.pixel_array[middlerow - int(np.floor(boxlength/2)):middlerow+int(np.floor(boxlength/2)),
                                                        middlecol-int(np.floor(boxlength/2)):middlecol+int(np.floor(boxlength/2))]
            rgbArray[:,:, 1] = zeropixels #dicom_data_g.pixel_array
            rgbArray[..., 2] = zeropixels #dicom_data_b.pixel_array
            #img = Image.fromarray(rgbArray)

   

            #
            # Plot the dicom with the annotation and the border of the subset
            #
            boxfig=plt.figure()
            
            im1=plt.imshow(dicom_data_r.pixel_array,cmap = 'gray',interpolation='none')
            plt.xlabel('Column (pixels)')
            plt.ylabel('Row (pixels)')
            plt.colorbar()
            
            # Get the current reference
            ax = plt.gca()
            
            # Create a Rectangle patch
            lowerx = middlecol-int(np.floor(boxlength/2))
            lowery = middlerow - int(np.floor(boxlength/2)) 
            rect = Rectangle((lowerx,lowery),boxlength,boxlength,linewidth=1,edgecolor='r',facecolor='none')
            # Add the patch to the Axes
            ax.add_patch(rect)
            

            #add in the annotated area on top of the raw data
            mask = np.ones((512,512))
            empty = np.ma.masked_where(mha_array[slice,:,:] >0,mask)
            mmm=mask.astype(np.float)
            #MHA values should be one for annotated areas, 0 for all other pixels
            mmm[np.where(mha_array[slice,:,:] < 1)]=np.nan
            
            #plt.figure()
            #plt.imshow(dicom_data_r.pixel_array,cmap='gray')
            ax.imshow(mmm,cmap = 'rainbow',vmin=0,vmax=1)
            #plt.title('new test')
            #plt.colorbar()
            #print('max of mmm is \n',np.max(mmm))
            #print title for plot using patient id
            line1 = str(patient_directories[patient_number] + ': Contrast: ' + phase_name)
            plt.title(line1 + '\nRaw image with annotation overlay')
            
            anno_file = os.path.join(patient_image_folder, 'annotation_' + str(key) 
                                    +'_' + "{:04n}".format(filenum) + '.jpg')
            boxfig.savefig(anno_file, dpi=90, bbox_inches='tight')
            boxfig.clf()
            plt.close()
            
            
            
            #
            # Image subsets
            #
            
            subset_file = os.path.join(patient_image_folder, 'subset_' + str(key) 
                                    +'_' + "{:04n}".format(filenum) + '.jpg')
            subfig = plt.figure()
            plt.imshow(rgbArray[:,:,0],cmap='bone')
            plt.colorbar()
            plt.xlabel('Column (pixels)')
            plt.ylabel('Row (pixels)')
            plt.title(line1 + '\nsubset image')
            subfig.savefig(subset_file, dpi=90, bbox_inches='tight')
            subfig.clf()
            plt.close()
            
            
            #
            # histogram across lesion middle
            #
            # we need the middle slice of the lesion and the row,col center of the lesion
            histo_file = os.path.join(patient_image_folder, 'histogram_' + str(key) 
                                    +'_' + "{:04n}".format(filenum) + '.jpg')
            print('lesion middle slice is \n',lesion_middle_slice)
            hfig = plt.figure()
            plt.hist(dicom_data_r.pixel_array[middlerow,:])
            plt.title(line1 + '\nHistogram across middle row of lesion')
            plt.xlabel('Image value bins')
            plt.ylabel('Frequency of values')
            print(dicom_data_r.pixel_array.shape)
            hfig.savefig(histo_file,dpi=90,bbox_inches ='tight')
            hfig.clf()
            plt.close()
            
            
            
            slice_file = os.path.join(patient_image_folder, 'data_slice_' + str(key) 
                                    +'_' + "{:04n}".format(filenum) + '.jpg')            
            slice_fig=plt.figure()
            row_data = dicom_data_r.pixel_array[middlerow,:]
            plt.plot(row_data) #dicom_data_r.pixel_array[middlerow,:])
            # draw vertical line from (70,100) to (70, 250)
            max_row_data = max(row_data)
            min_row_data = min(row_data)
            plt.plot([middlecol, middlecol], [min_row_data,max_row_data], 'r-', lw=2)
            plt.ylabel('Raw Image Value')
            plt.xlabel('column (pixel)')
            plt.title(line1 + '\Slice through the center of annotated lesion')
            slice_fig.savefig(slice_file,dpi=90,bbox_inches='tight')
            slice_fig.clf()
            plt.close('all')
            

            gc.collect() #garbage collection after closing figs


print('FINISHED PROCESSING IMAGES\n')     
return

<class 'list'>
entering main section
Slice location...: -90.0000000000
Slice location...: -93.0000000000
Slice location...: -96.0000000000
Slice location...: -99.0000000000
Slice location...: -102.0000000000
Slice location...: -105.0000000000
Slice location...: -108.0000000000
Slice location...: -111.0000000000
Slice location...: -114.0000000000
Slice location...: -117.0000000000
Slice location...: -120.0000000000
Slice location...: -123.0000000000
Slice location...: -126.0000000000
Slice location...: -129.0000000000
Slice location...: -132.0000000000
Slice location...: -135.0000000000
Slice location...: -138.0000000000
Slice location...: -141.0000000000
Slice location...: -144.0000000000
Slice location...: -147.0000000000
Slice location...: -150.0000000000
Slice location...: -153.0000000000
Slice location...: -156.0000000000
Slice location...: -159.0000000000
Slice location...: -162.0000000000
Slice location...: -165.0000000000
Slice location...: -168.0000000000
Slice location...: -17

Slice location...: 139.5
Slice location...: 142.5
Slice location...: 145.5
Slice location...: 148.5
Slice location...: 151.5
Slice location...: 154.5
Slice location...: 157.5
Slice location...: 160.5
Slice location...: 163.5
Slice location...: 166.5
Slice location...: 169.5
Slice location...: 172.5
Slice location...: 175.5
Slice location...: 178.5
Slice location...: 181.5
Slice location...: 184.5
Slice location...: 187.5
Slice location...: 190.5
Slice location...: 193.5
Slice location...: 196.5
Slice location...: 199.5
Slice location...: 202.5
Slice location...: 205.5
Slice location...: 208.5
Slice location...: 211.5
Slice location...: 214.5
Slice location...: 217.5
Slice location...: 220.5
Slice location...: 223.5
Slice location...: 226.5
Slice location...: 229.5
Slice location...: 232.5
Slice location...: 235.5
Slice location...: 238.5
[45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55]
Directory  /media/kgonzalez/Seagate Backup Plus Drive/PROJECT_PLOTS/RCC/Post_Exc/10024-0014_2006-06-16  C

Slice location...: 329
Slice location...: 332
Slice location...: 335
Slice location...: 338
Slice location...: 341
Slice location...: 344
Slice location...: 347
Slice location...: 350
[39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53]
Directory  /media/kgonzalez/Seagate Backup Plus Drive/PROJECT_PLOTS/RCC/Post_Exc/10024-0016_2007-04-20  Created 
filenumber is now  39  out of  101
middle row, col
 309 168
lesion middle slice is 
 46
(512, 512)
filenumber is now  40  out of  101
middle row, col
 309 168
lesion middle slice is 
 46
(512, 512)
filenumber is now  41  out of  101
middle row, col
 309 168
lesion middle slice is 
 46
(512, 512)
filenumber is now  42  out of  101
middle row, col
 309 168
lesion middle slice is 
 46
(512, 512)
filenumber is now  43  out of  101
middle row, col
 309 168
lesion middle slice is 
 46
(512, 512)
filenumber is now  44  out of  101
middle row, col
 309 168
lesion middle slice is 
 46
(512, 512)
filenumber is now  45  out of  101
middle row, c

filenumber is now  54  out of  78
middle row, col
 257 406
lesion middle slice is 
 53
(512, 512)
filenumber is now  55  out of  78
middle row, col
 257 406
lesion middle slice is 
 53
(512, 512)
Slice location...: 162
Slice location...: 165
Slice location...: 168
Slice location...: 171
Slice location...: 174
Slice location...: 177
Slice location...: 180
Slice location...: 183
Slice location...: 186
Slice location...: 189
Slice location...: 192
Slice location...: 195
Slice location...: 198
Slice location...: 201
Slice location...: 204
Slice location...: 207
Slice location...: 210
Slice location...: 213
Slice location...: 216
Slice location...: 219
Slice location...: 222
Slice location...: 225
Slice location...: 228
Slice location...: 231
Slice location...: 234
Slice location...: 237
Slice location...: 240
Slice location...: 243
Slice location...: 246
Slice location...: 249
Slice location...: 252
Slice location...: 255
Slice location...: 258
Slice location...: 261
Slice location...: 264

Slice location...: 188
Slice location...: 191
Slice location...: 194
Slice location...: 197
Slice location...: 200
Slice location...: 203
Slice location...: 206
Slice location...: 209
Slice location...: 212
Slice location...: 215
Slice location...: 218
Slice location...: 221
Slice location...: 224
Slice location...: 227
Slice location...: 230
Slice location...: 233
Slice location...: 236
Slice location...: 239
Slice location...: 242
Slice location...: 245
Slice location...: 248
Slice location...: 251
Slice location...: 254
Slice location...: 257
Slice location...: 260
Slice location...: 263
Slice location...: 266
Slice location...: 269
Slice location...: 272
Slice location...: 275
Slice location...: 278
Slice location...: 281
Slice location...: 284
Slice location...: 287
Slice location...: 290
Slice location...: 293
Slice location...: 296
Slice location...: 299
Slice location...: 302
Slice location...: 305
Slice location...: 308
Slice location...: 311
Slice location...: 314
Slice locat

...

Slice location...: 555
Slice location...: 558
Slice location...: 561
Slice location...: 564
Slice location...: 567
[38, 39, 40, 41, 42, 43, 44, 45, 46, 47]
Directory  /media/kgonzalez/Seagate Backup Plus Drive/PROJECT_PLOTS/RCC/Post_Exc/10024-0027_2006-05-31  Created 
filenumber is now  38  out of  126
middle row, col
 268 156
lesion middle slice is 
 42
(512, 512)
filenumber is now  39  out of  126
middle row, col
 268 156
lesion middle slice is 
 42
(512, 512)
filenumber is now  40  out of  126
middle row, col
 268 156
lesion middle slice is 
 42
(512, 512)
filenumber is now  41  out of  126
middle row, col
 268 156
lesion middle slice is 
 42
(512, 512)
filenumber is now  42  out of  126
middle row, col
 268 156
lesion middle slice is 
 42
(512, 512)
filenumber is now  43  out of  126
middle row, col
 268 156
lesion middle slice is 
 42
(512, 512)
filenumber is now  44  out of  126
middle row, col
 268 156
lesion middle slice is 
 42
(512, 512)
filenumber is now  45  out of  126
mid

Slice location...: -42.5999984741
Slice location...: -49.5999984741
Slice location...: -56.5999984741
Slice location...: -63.5999984741
Slice location...: -70.5999984741
Slice location...: -77.5999984741
Slice location...: -84.5999984741
Slice location...: -91.5999984741
Slice location...: -98.5999984741
Slice location...: -105.5999984741
Slice location...: -112.5999984741
Slice location...: -119.5999984741
Slice location...: -126.5999984741
Slice location...: -133.6000061035
Slice location...: -140.6000061035
Slice location...: -147.6000061035
Slice location...: -154.6000061035
Slice location...: -161.6000061035
Slice location...: -168.6000061035
Slice location...: -175.6000061035
Slice location...: -182.6000061035
Slice location...: -189.6000061035
Slice location...: -196.6000061035
Slice location...: -203.6000061035
Slice location...: -210.6000061035
Slice location...: -217.6000061035
Slice location...: -224.6000061035
Slice location...: -231.6000061035
Slice location...: -238.60000

Slice location...: 196
Slice location...: 199
Slice location...: 202
Slice location...: 205
Slice location...: 208
Slice location...: 211
Slice location...: 214
Slice location...: 217
Slice location...: 220
Slice location...: 223
Slice location...: 226
Slice location...: 229
Slice location...: 232
Slice location...: 235
Slice location...: 238
Slice location...: 241
Slice location...: 244
Slice location...: 247
Slice location...: 250
Slice location...: 253
Slice location...: 256
Slice location...: 259
Slice location...: 262
Slice location...: 265
Slice location...: 268
Slice location...: 271
Slice location...: 274
Slice location...: 277
Slice location...: 280
Slice location...: 283
Slice location...: 286
Slice location...: 289
Slice location...: 292
Slice location...: 295
Slice location...: 298
Slice location...: 301
Slice location...: 304
Slice location...: 307
Slice location...: 310
Slice location...: 313
Slice location...: 316
Slice location...: 319
Slice location...: 322
Slice locat

Slice location...: 257.7
Slice location...: 258.3
Slice location...: 258.9
Slice location...: 259.5
Slice location...: 260.1
Slice location...: 260.7
Slice location...: 261.3
Slice location...: 261.9
Slice location...: 262.5
Slice location...: 263.1
Slice location...: 263.7
Slice location...: 264.3
Slice location...: 264.9
Slice location...: 265.5
Slice location...: 266.1
Slice location...: 266.7
Slice location...: 267.3
Slice location...: 267.9
Slice location...: 268.5
Slice location...: 269.1
Slice location...: 269.7
Slice location...: 270.3
Slice location...: 270.9
Slice location...: 271.5
Slice location...: 272.1
Slice location...: 272.7
Slice location...: 273.3
Slice location...: 273.9
Slice location...: 274.5
Slice location...: 275.1
Slice location...: 275.7
Slice location...: 276.3
Slice location...: 276.9
Slice location...: 277.5
Slice location...: 278.1
Slice location...: 278.7
Slice location...: 279.3
Slice location...: 279.9
Slice location...: 280.5
Slice location...: 281.1


KeyboardInterrupt: 

In [ ]:
mask = np.zeros((10,10))
#mask[3:-3, 3:-3] = 1 # white square in black background
mask[0,0]=1
im = mask + np.random.randn(10,10) * 0.01 # random image
masked = np.ma.masked_where(mask == 0, mask)
print('mask type is ',type(masked))
print(masked)


empty = np.ma.masked_where(im >0,mask)
print(empty)
print(empty.data)
plt.figure()
plt.imshow(empty.data,cmap='gray')
plt.imshow(empty,cmap='jet')
plt.figure()
plt.imshow(im,cmap='gray')
mmm=mask.astype(np.float)
mmm[np.where(empty.data==0)]=np.nan
plt.imshow(mmm,cmap = 'rainbow',vmin=0,vmax=1)


In [ ]:
a = 'test' + '\n' + 'testline 2'
print(a)